<a href="https://colab.research.google.com/github/Harsshadpawar06/GenerativeAI/blob/main/MachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
print("ok!")

ok!


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 25 07:06:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [8]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.27.2
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
be

In [9]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [10]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [11]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

# own translation system
# language translation system
# from the hugging face we are going to use open source translation model
# data set also i wll take from hugging face

#1. Transfer learning
#2. FineTuning (for this translation system we gonna perform finetuning of pretrain model)
#3. RAG System (DO Not Focus here)

Google - T5

Facebook - M2M100

Google - Google Cloud Translation API

Tune AI

Amazone - Amazon COmprehend API

In [12]:
model= "Helsinki-NLP/opus-mt-en-hi"

In [13]:
dataset= load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [15]:
dataset["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [16]:
!pip install sacremoses

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [18]:
tokenizer("there is sunny who is teaching data science")

{'input_ids': [188, 23, 41670, 66, 23, 1762, 1195, 7373, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenizer(["there is Harshad who is teaching Deep learning","there is sunny who is teaching data science"])

{'input_ids': [[188, 23, 11846, 16, 14487, 66, 23, 1762, 19200, 3729, 0], [188, 23, 41670, 66, 23, 1762, 1195, 7373, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [20]:
tokenizer(['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'])

{'input_ids': [[44, 4499, 2703, 314, 174, 44, 4499, 314, 3625, 2703, 1185, 428, 1321, 917, 3355, 44, 716, 917, 44, 2703, 24217, 549, 4228, 314, 130, 12604, 260, 44, 926, 1185, 1321, 260, 1321, 260, 1056, 44, 716, 260, 44, 800, 260, 7817, 44, 13087, 549, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [21]:
with tokenizer.as_target_tokenizer():
    labels = tokenizer("अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [22]:
print(labels)

{'input_ids': [63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [23]:
dataset['train']['translation'][0:10]

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

In [24]:
for  ex in dataset['train']['translation'][0:10]:
  print(ex['en'])

Give your application an accessibility workout
Accerciser Accessibility Explorer
The default plugin layout for the bottom panel
The default plugin layout for the top panel
A list of plugins that are disabled by default
Highlight duration
The duration of the highlight box when selecting accessible nodes
Highlight border color
The color and opacity of the highlight border.
Highlight fill color


In [25]:
for  ex in dataset['train']['translation'][0:10]:
  print(ex['hi'])

अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
एक्सेर्साइसर पहुंचनीयता अन्वेषक
निचले पटल के लिए डिफोल्ट प्लग-इन खाका
ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
अवधि को हाइलाइट रकें
पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि
सीमांत (बोर्डर) के रंग को हाइलाइट करें
हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। 
भराई के रंग को हाइलाइट करें


In [26]:
max_input_length = 128
max_target_length = 128

source_lang="en"
target_lang="hi"

def process_function(sentence):

  inputs=[ex[source_lang] for ex in sentence["translation"]]
  targets=[ex[target_lang] for ex in sentence["translation"]]

  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)


  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [27]:
tokenized_data=dataset.map(process_function, batched=True)

In [28]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [29]:
model

'Helsinki-NLP/opus-mt-en-hi'

In [30]:
from transformers import TFAutoModelForSeq2SeqLM
model=TFAutoModelForSeq2SeqLM.from_pretrained(model)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [31]:
batch_size=32
learning_rate=0.001
weight_decay=0.01
num_train_epochs=1

In [32]:
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [33]:
train_dataset=model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator

)

In [34]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,

)

In [35]:
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

In [36]:
model.compile(optimizer=optimizer)

In [37]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=num_train_epochs)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
78/78 [==============================] - 118s 412ms/step - loss: 4.5773 - val_loss: 4.7926


In [38]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


# model inferencing

In [45]:
input_text="i am learning coding. how about your coding"

In [46]:
tokenized=tokenizer([input_text],return_tensors='np')

In [47]:
tokenized

{'input_ids': array([[5556,  489, 3729, 7308,  232,  137,    3,  287,  195,   85, 7308,
         232,  137,    0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
output=model.generate(**tokenized, max_length=128)

In [53]:
with tokenizer.as_target_tokenizer():
  decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_text)

मैं अपने समूह के बारे में कैसे हूँ।
